## Exercise: OPERA-$\nu$ oscillations  
Discovery of tau neutrino appearance in the CNGS neutrino beam with the OPERA experiment

#### PART 1: Counting Model in RooFit (one channel) with uncertain background

Let's suppose we observe nobs events when we expect nexp, where nexp = µ s+b (µ is the signal strength, s is the nominal number of signal events and b is the number of background events. The expected distribution for nexp is a Poisson Poisson(nobs | µ s +b). µ is the parameter we want to estimate or set a limit (the parameter of interest), while b is the nuisance parameter. The real value of b is unknown. Its best estimate is "b0" with uncertainty sigmab. To express this uncertainty we add in the model a Gaussian constraint. We can interpret this term as having an additional measurement b0 with an uncertainty sigmab: i.e. we have a likelihood for that measurement Gaussian( b0 | b, sigmab).

In [1]:
import ROOT

In [2]:
#### Create a counting model with signal strength based on Poisson Statistics ####

# declare the observable with a range
nobs = ROOT.RooRealVar("nobs", "number of observed events", 0, 10)

# true, unknown number of signal events
mu = ROOT.RooRealVar("mu", "signal strength", 1, 0, 5)
nsig = ROOT.RooRealVar("nsig", "nominal number of signal events", 2.64)  # constant value taken from the paper

# true, unknown number of background events
nbkg = ROOT.RooRealVar("nbkg", "expected number of background events", 1, 0, 10)  # parameter

# true, unknown number of total events
nexp = ROOT.RooFormulaVar("nexp", "@0 * @1 + @2", ROOT.RooArgList(mu, nsig, nbkg))

# Poisson(nobs|mu*nsig + nbkg)
pois = ROOT.RooPoisson("pois", "Poisson model", nobs, nexp)

In [3]:
#### Create a Gaussian PDF to constraint the background ####
# From Table 3 of the paper, the total background: 0.25 ± 0.05

sigma = ROOT.RooRealVar("sigma", "width of gaussian", 0.05)  # constant
nbkg_estimate = ROOT.RooRealVar("nbkg_estimate", "global observable", 0.25, 0, 10)
# mean is already defined = nbkg

# gaussian pdf
gaus = ROOT.RooGaussian("gaus", "Gaussian model", nbkg_estimate, nbkg, sigma)

[#0] WARNING:InputArguments -- The parameter 'sigma' with range [-inf, inf] of the RooGaussian 'gaus' exceeds the safe range of (0, inf). Advise to limit its range.


In [4]:
# total probability model
model = ROOT.RooProdPdf("model", "1 channel number-counting model with constraints", ROOT.RooArgList(pois, gaus))

We generate a hypothetical observed data set. Since we have a counting model, the data set will contain only one event and the observable nobs will have our desired value .

In [5]:
# make data set with the number of observed events
data = ROOT.RooDataSet("data", "", nobs)
nobs.setVal(5)
data.add(nobs)

Import the model and the data in a RooFit RooWorkspace and save to file

In [6]:
# import dataset in a workspace
w = ROOT.RooWorkspace("w")
w.Import(model)
w.Import(data)

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooProdPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooPoisson::pois
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nobs
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooFormulaVar::nexp
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mu
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nsig
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nbkg
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::gaus
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nbkg_estimate
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::sigma
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset data


In [7]:
w.Print()


RooWorkspace(w) w contents

variables
---------
(mu,nbkg,nbkg_estimate,nobs,nsig,sigma)

p.d.f.s
-------
RooGaussian::gaus[ x=nbkg_estimate mean=nbkg sigma=sigma ] = 1.38634e-49
RooProdPdf::model[ pois * gaus ] = 1.93805e-50
RooPoisson::pois[ x=nobs mean=nexp ] = 0.139796

functions
--------
RooFormulaVar::nexp[ actualVars=(mu,nsig,nbkg) formula="@0 * @1 + @2" ] = 3.64

datasets
--------
RooDataSet::data(nobs)



Create the ModelConfig object and import in the workspace. We need to add in the ModelConfig also b0 as a "global observable". The reason is that b0 needs to be treated as an auxiliary observable in case of frequentist statistics and varied when tossing pseudo-experiments.

In [8]:
mc = ROOT.RooStats.ModelConfig("model_config")
mc.SetWorkspace(w)
mc.SetPdf(w.pdf("model"))
mc.SetParametersOfInterest(w.var("mu"))
mc.SetObservables(w.var("nobs"))

# the estimate of gaussian constrained by a gaussian
mc.SetGlobalObservables(w.var("nbkg_estimate"))
w.var("nbkg_estimate").setConstant(True)  # needed when we fit the model

# all the other variables are nuisance parameters
mc.SetNuisanceParameters(w.var("nbkg"))

mc.Print()

# import to the workspace
w.Import(mc)

False


=== Using the following for model_config ===
Observables:             RooArgSet:: = (nobs)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (nbkg)
Global Observables:      RooArgSet:: = (nbkg_estimate)
PDF:                     RooProdPdf::model[ pois * gaus ] = 1.93805e-50



In [9]:
# save the workspace to a file
w.writeToFile("CountingModel.root", True)

False

To estimate the significance, we need to perform an hypothesis test. We want to disprove the null model, i.e the background only model against the alternate model, the background plus the signal. In RooStats, we do this by defining two ModelConfig objects, one for the null model (the background only model in this case) and one for the alternate model (the signal plus background).

The null hypothesis (i.e. the observed number of events is just a statistical fluctuation of the background only) corresponds to a signal strength mu = 0.

In [10]:
# create the class using data and model config
plc = ROOT.RooStats.ProfileLikelihoodCalculator(data, mc)
# set value of POI to mu = 0
w.var("mu").setVal(0)
plc.SetNullParameters(w.var("mu"))
hypotest = plc.GetHypoTest()
alpha = hypotest.NullPValue()
significance = hypotest.Significance()

print(f"Significance: {significance:.3f} \n")

Significance: 4.441 

[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.
[#1] INFO:Minimization --  Including the following constraint terms in minimization: (gaus)
[#1] INFO:Minimization -- The following global observables have been defined and their values are taken from the model: (nbkg_estimate)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_data) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 /  with strategy 1
[#1] INFO:Minimization -- 
  RooFitRes

In [11]:
# set the confidence level to 68%
plc.SetConfidenceLevel(0.68)
# compute the interval of the parameter mu
interval = plc.GetInterval()
# print the interval (get a pointer of mu from mc)
poi = mc.GetParametersOfInterest().first()
lowerlimit = interval.LowerLimit(poi)
upperlimit = interval.UpperLimit(poi)

# plot the interval
plot = ROOT.RooStats.LikelihoodIntervalPlot(interval)

c = ROOT.TCanvas()
plot.Draw()
c.Update
plot.GetPlottedObject().GetYaxis().SetRangeUser(0, 10)
c.Draw()

[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.
[#1] INFO:Minimization --  Including the following constraint terms in minimization: (gaus)
[#1] INFO:Minimization -- The following global observables have been defined and their values are taken from the model: (nbkg_estimate)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_data) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 /  with strategy 1
[#1] INFO:Minimization -- 
  RooFitResult: minimized FCN value: -0.35123, estimated distance to minimum: 1.66159e-15
                covari

#### Method 3: using the ready-to-use RooStats [Standard Demo tutorials](https://root.cern.ch/doc/master/group__tutorial__roostats.html)

```js
(base) rianashaba@Rianas-MacBook-Pro ~ % eval "$(/Users/rianashaba/miniforge3/bin/conda shell.zsh hook)"
(base) rianashaba@Rianas-MacBook-Pro ~ % cd ROOT
(base) rianashaba@Rianas-MacBook-Pro ROOT % conda activate root_env
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % ls
CountingModel.root			higgs_4l.dat
Exercise11_Ad.ipynb			live_coding_session3_part1.ipynb
Exercise12_2.ipynb			minos_2013_data.dat
Exercise13_1.ipynb			minos_2013_mc.dat
Exercise13_2.ipynb			minos_data.png
Exercise2021_1.ipynb			rarest_b0_decay.dat
Exercise2021_2.ipynb			sess1.ipynb
ExerciseA.ipynb				session1.ipynb
ExerciseATLAS.ipynb			session2_1.ipynb
ExerciseOPERA.ipynb			session2_2.ipynb
StandardFeldmanCousinDemo.C		session4_1.ipynb
StandardHypoTestDemo.C			test.root
atlas_4l.root				test.rtf
b0_invariant_mass.root			tutorials
exercise1.ipynb
(root_env) rianashaba@Rianas-MacBook-Pro ROOT % root CountingModel.root 
   ------------------------------------------------------------------
  | Welcome to ROOT 6.34.04                        https://root.cern |
  | (c) 1995-2024, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for macosxarm64 on Feb 26 2025, 15:56:26                   |
  | From tags/6-34-04@6-34-04                                        |
  | With                                                             |
  | Try '.help'/'.?', '.demo', '.license', '.credits', '.quit'/'.q'  |
   ------------------------------------------------------------------

root [0] 
Attaching file CountingModel.root as _file0...
(TFile *) 0x11de97690
root [1] .ls
TFile**		CountingModel.root	
 TFile*		CountingModel.root	
  KEY: RooWorkspace	w;1	w
  KEY: TProcessID	ProcessID0;1	4af9d8c0-d42e-11f0-b846-020a14acbeef
root [2] w
(RooWorkspace *) 0x10c80ce00
root [3] w->Print()

RooWorkspace(w) w contents

variables
---------
(mu,nbkg,nbkg_estimate,nobs,nsig,sigma)

p.d.f.s
-------
RooGaussian::gaus[ x=nbkg_estimate mean=nbkg sigma=sigma ] = 1.38634e-49
RooProdPdf::model[ pois * gaus ] = 1.93805e-50
RooPoisson::pois[ x=nobs mean=nexp ] = 0.139796

functions
--------
RooFormulaVar::nexp[ actualVars=(mu,nsig,nbkg) formula="@0 * @1 + @2" ] = 3.64

datasets
--------
RooDataSet::data(nobs)

named sets
----------
model_config_GlobalObservables:(nbkg_estimate)
model_config_NuisParams:(nbkg)
model_config_Observables:(nobs)
model_config_POI:(mu)

generic objects
---------------
RooStats::ModelConfig::model_config

root [4] .L $ROOTSYS/tutorials/roo
roofit/
roostats/
rootlogoff.C
rootlogon.C
root [4] .L $ROOTSYS/tutorials/roostats/StandardHypoTestDemo.C 
root [5] StandardHypoTestDemo(
void StandardHypoTestDemo(const char* infile = "", const char* workspaceName = "combined", const char* modelSBName = "ModelConfig", const char* modelBName = "", const char* dataName = "obsData", int calcType = 0, int testStatType = 3, int ntoys = 5000, bool useNC = false, const char* nuisPriorName = 0)
root [5] StandardHypoTestDemo("/Users/rianashaba/ROOT/CountingModel.root", "w", "model_config", "", "data", 2, 3, 0, true)

RooWorkspace(w) w contents

variables
---------
(mu,nbkg,nbkg_estimate,nobs,nsig,sigma)

p.d.f.s
-------
RooGaussian::gaus[ x=nbkg_estimate mean=nbkg sigma=sigma ] = 1.38634e-49
RooProdPdf::model[ pois * gaus ] = 1.93805e-50
RooPoisson::pois[ x=nobs mean=nexp ] = 0.139796

functions
--------
RooFormulaVar::nexp[ actualVars=(mu,nsig,nbkg) formula="@0 * @1 + @2" ] = 3.64

datasets
--------
RooDataSet::data(nobs)

named sets
----------
model_config_GlobalObservables:(nbkg_estimate)
model_config_NuisParams:(nbkg)
model_config_Observables:(nobs)
model_config_POI:(mu)

generic objects
---------------
RooStats::ModelConfig::model_config

Info in <StandardHypoTestInvDemo>: The background model  does not exist
Info in <StandardHypoTestInvDemo>: Copy it from ModelConfig model_config and set POI to zero
Info in <StandardHypoTestDemo>: Model model_config has no snapshot  - make one using model poi
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize - Find  best unconditional NLL on observed data
[#0] PROGRESS:Eval -- Best fitted POI value = 1.84289 +/- 0.894172
[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set
[#1] INFO:InputArguments -- AsymptoticCalculator: Asimov data will be generated using fitted nuisance parameter values
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize Find  best conditional NLL on ASIMOV data set for given alt POI ( mu ) = 1

Results HypoTestAsymptotic_result: 
 - Null p-value = 4.47597e-06
 - Significance = 4.44105
 - CL_b: 4.47597e-06
 - CL_s+b: 0.0654907
 - CL_s: 14631.6
Asymptotic results 
 Expected p -value and significance at -2 sigma = 0.175976 significance 0.930811 sigma 
 Expected p -value and significance at -1 sigma = 0.0267532 significance 1.93081 sigma 
 Expected p -value and significance at 0 sigma = 0.00169039 significance 2.93081 sigma 
 Expected p -value and significance at 1 sigma = 4.23299e-05 significance 3.93081 sigma 
 Expected p -value and significance at 2 sigma = 4.09445e-07 significance 4.93081 sigma 
root [6] 
```